A simple Travelling Salesman Problem to solve (optimize) with IBM Cplex for learning purposes

_"Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?"_

*(Wikipedia.org, 2023)* - https://en.wikipedia.org/wiki/Travelling_salesman_problem

![problem-studied](./documentation/problem.png)

In [7]:
#0. IMPORT ALL USED LIBRARIES
from random import randint
import numpy as np

In [6]:
# I. BUILD A RANDOM TSP INSTACE 
def build_random_instance(nbr_cities=6, min_distance=1, max_distance=15):
    start = randint(0, nbr_cities-1) # 1. random generate the starting city
    distances = np.random.uniform(min_distance, max_distance, (nbr_cities, nbr_cities)) # 2. random generate a matrix of paths between cities
    distances = np.triu(distances, k=1)
    distances += distances.T # 3. the distance between city a -> city b should be equal to the between city b -> city a
    np.fill_diagonal(distances, 0) # 4. replace all distance for a city to itself (the diagonal) to 0 
    return distances, start

distances, start = build_random_instance(4,1,10)
print(distances)
print(start)

[[0.         3.62309714 2.68463527 5.03271522]
 [3.62309714 0.         6.31097702 8.1110366 ]
 [2.68463527 6.31097702 0.         5.34263084]
 [5.03271522 8.1110366  5.34263084 0.        ]]
2


In [8]:
#II. BUILD THE CPLEX MODEL

![calcul-of-subsets](./documentation/calcul.png)